# Has One Has Many Lab

### Introduction

### Our Task

In [1]:
import sqlite3
billboard_conn = sqlite3.connect('./billboard.db')
cursor = billboard_conn.cursor()

In [2]:
cursor.execute('SELECT name from sqlite_master where type= "table"')
cursor.fetchall()

[('albums',), ('acoustic_features',)]

In [3]:
cursor.execute('PRAGMA table_info(albums);')
cursor.fetchall()

[(0, 'id', 'integer', 0, None, 1),
 (1, 'date', 'text', 0, None, 0),
 (2, 'artist', 'text', 0, None, 0),
 (3, 'album', 'text', 0, None, 0),
 (4, 'rank', 'text', 0, None, 0),
 (5, 'length', 'integer', 0, None, 0),
 (6, 'track_length', 'real', 0, None, 0)]

Let's begin to normalize our albums table above.  Now we discussed in the last lesson that artist should be separated out into a different table.  So what we should do is the following.  First we should create an artist's table that has an name and artist id.  

Then we need to populate the table with each of the artists.

#### Creating the artists table

Let's select our data and see how far away we are.

In [4]:
cursor.execute('SELECT DISTINCT(artist) from albums;')
results = cursor.fetchall()

In [7]:
results[0:3]

[(None,), ('A Boogie Wit da Hoodie',), ('21 Savage',)]

Looks like we don't need that first result.

In [6]:
results[1:3]

[('A Boogie Wit da Hoodie',), ('21 Savage',)]

Now instead of using purely sql to insert our data, the easiest way is to use just a couple of lines from a tool called pandas.

In [8]:
import pandas as pd
artists_df = pd.DataFrame(results[1:], columns = ['name'], index = range(1, len(results)))

In [10]:
artists_df[:3]

,name
1,A Boogie Wit da Hoodie
2,21 Savage
3,Soundtrack


In the above lines we converted our list of tuples into a dataframe.  We named our column `name`, and we specified that our index (the list of numbers to the left should go from 1 to the length of the results).

Now the dataframe thing isn't that important for now.  The reason why we use it, is because we can go from dataframes to adding another sql table.  Let's see it.

In [21]:
new_billboard_conn = sqlite3.connect('./new_billboard.db')
new_cursor = new_billboard_conn.cursor()

artists_df.to_sql('artists', billboard_conn, index=True, index_label='id')

That did the trick.  

So we used our `dataframe` to then call `to_sql`.  We have to specify the table name we would like to be created from our dataframe, and then where we would likek to place this table -- the database represented by our `conn` object.  Then because we would like a column for the id, we say to insert the data from the index, and label that column as `id`.

Let's see how we did.

In [22]:
cursor.execute('SELECT name from sqlite_master where type= "table"')
cursor.fetchall()

# [('albums',), ('acoustic_features',), ('artists',), ('sqlite_sequence',)]

[('albums',), ('acoustic_features',), ('artists',)]

So we see that we now have a new table called artists.  Let's make sure that the columns of that table are set up correctly.

In [23]:
cursor.execute('PRAGMA table_info(artists)')
cursor.fetchall()

# [(0, 'id', 'INTEGER', 0, None, 1), (1, 'name', 'TEXT', 0, None, 0)]

[(0, 'id', 'INTEGER', 0, None, 0), (1, 'name', 'TEXT', 0, None, 0)]

Great! And now let's just look at a sample of the data.

In [24]:
new_cursor.execute('SELECT * from artists limit 3')
new_cursor.fetchall()

[(1, 'A Boogie Wit da Hoodie'), (2, '21 Savage'), (3, 'Soundtrack')]

### Updating the albums table

Now taking another look at the albums table we can see that we have an extra data in there.  We would like to create a new table that does not have the `artist`, but the artist id.

In [25]:
cursor.execute('PRAGMA table_info(albums)')
cursor.fetchall()

[(0, 'id', 'integer', 0, None, 1),
 (1, 'date', 'text', 0, None, 0),
 (2, 'artist', 'text', 0, None, 0),
 (3, 'album', 'text', 0, None, 0),
 (4, 'rank', 'text', 0, None, 0),
 (5, 'length', 'integer', 0, None, 0),
 (6, 'track_length', 'real', 0, None, 0)]

In [26]:
cursor.execute('SELECT artists.id, artists.name, albums.* FROM albums INNER JOIN artists ON albums.artist = artists.name;')

In [27]:
results = cursor.fetchall()

In [28]:
results[0:2]

[(1,
  'A Boogie Wit da Hoodie',
  2,
  '2019-01-19',
  'A Boogie Wit da Hoodie',
  'Hoodie SZN',
  '1',
  20,
  185233.8),
 (2,
  '21 Savage',
  3,
  '2019-01-19',
  '21 Savage',
  'I Am > I Was',
  '2',
  15,
  211050.73333333334)]

In [29]:
df_columns = ['artist_id', 'artist_name', 'album_id', 'date', 'artist', 'name', 'rank', 'length', 'track_length']
albums_joined_df = pd.DataFrame(results, columns = df_columns, index = range(1, len(results) + 1) )

In [30]:
df_albums_limited = albums_joined_df[['name', 'artist_id', 'date', 'rank', 'length', 'track_length']]

In [31]:
df_albums_limited[0:3]

,name,artist_id,date,rank,length,track_length
1,Hoodie SZN,1,2019-01-19,1,20.0,185233.800000
2,I Am > I Was,2,2019-01-19,2,15.0,211050.733333
3,Spider-Man: Into The Spider-Verse,3,2019-01-19,3,13.0,190866.384615


In [32]:
cursor.execute('DROP TABLE albums;')

In [33]:
cursor.execute('SELECT name from sqlite_master where type= "table"')
cursor.fetchall()

[('acoustic_features',), ('artists',)]

In [35]:
df_albums_limited.to_sql('albums', billboard_conn, index = True, index_label = 'id')

In [36]:
cursor.execute('SELECT name from sqlite_master where type= "table"')
cursor.fetchall()

[('acoustic_features',), ('artists',), ('albums',)]

In [11]:
cursor.execute('PRAGMA table_info(albums)')
album_cols = cursor.fetchall()

In [12]:
album_column_names = [col_pair[1] for col_pair in album_cols]

### Reducing the acoustic features table

In [8]:
cursor.execute('PRAGMA table_info(acoustic_features)')
acoustic_table_columns = cursor.fetchall()

In [9]:
column_names = [col_pair[1] for col_pair in acoustic_table_columns]

In [56]:
# column_names

* Now for the line below, be prepared to wait about ten minutes.  That's just the cost of doing business.

In [5]:
cursor.execute('select albums.*, acoustic_features.* from acoustic_features join albums on albums.name = acoustic_features.album;')
acoustic_table_data = cursor.fetchall()

In [6]:
len(acoustic_table_data[0])

26

In [42]:
album_column_names = ['alb_id', 'name', 'artist_id', 'alb_date', 'rank', 'length', 'track_length']

In [43]:
acoustic_cols = album_column_names + column_names 

In [44]:
import pandas as pd
df_acoustic_few = pd.DataFrame(acoustic_table_data[:3], columns = acoustic_cols)

In [45]:
dtypes = df_acoustic_few.dtypes.to_dict()

In [46]:
len(df_acoustic_few.dtypes)

26

In [47]:
df_acoustic = pd.DataFrame(acoustic_table_data, columns = dtypes)

In [49]:
df_acoustic_columns = ['id', 'song', 'alb_id', 'acousticness',
       'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key',
       'liveness', 'loudness', 'mode', 'speechiness', 'tempo',
       'time_signature', 'valence', 'date']

In [50]:
selected_df_acoustic = df_acoustic[df_acoustic_columns]

In [51]:
selected_df_acoustic.columns = ['id', 'song', 'album_id', 'acousticness',
       'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key',
       'liveness', 'loudness', 'mode', 'speechiness', 'tempo',
       'time_signature', 'valence', 'date']

In [53]:
reduced_df_acoustic = selected_df_acoustic[['song', 'album_id', 'acousticness',
       'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key',
       'liveness', 'loudness', 'mode', 'speechiness', 'tempo',
       'time_signature', 'valence', 'date']]

In [55]:
reduced_df_acoustic.to_sql('songs', billboard_conn, index=True, index_label='id')

In [57]:
cursor.execute('SELECT name from sqlite_master where type= "table"')
cursor.fetchall()

[('acoustic_features',), ('artists',), ('albums',), ('songs',)]

In [59]:
cursor.execute('PRAGMA table_info(songs);')
cursor.fetchall()

[(0, 'id', 'INTEGER', 0, None, 0),
 (1, 'song', 'TEXT', 0, None, 0),
 (2, 'album_id', 'INTEGER', 0, None, 0),
 (3, 'acousticness', 'REAL', 0, None, 0),
 (4, 'danceability', 'REAL', 0, None, 0),
 (5, 'duration_ms', 'REAL', 0, None, 0),
 (6, 'energy', 'REAL', 0, None, 0),
 (7, 'instrumentalness', 'REAL', 0, None, 0),
 (8, 'key', 'REAL', 0, None, 0),
 (9, 'liveness', 'REAL', 0, None, 0),
 (10, 'loudness', 'REAL', 0, None, 0),
 (11, 'mode', 'REAL', 0, None, 0),
 (12, 'speechiness', 'REAL', 0, None, 0),
 (13, 'tempo', 'REAL', 0, None, 0),
 (14, 'time_signature', 'REAL', 0, None, 0),
 (15, 'valence', 'REAL', 0, None, 0),
 (16, 'date', 'TEXT', 0, None, 0)]

In [60]:
cursor.execute('DROP TABLE acoustic_features;')

In [61]:
cursor.execute('SELECT name from sqlite_master where type= "table"')
cursor.fetchall()

[('artists',), ('albums',), ('songs',)]